In [1]:
cd /Users/emilyvincett/Downloads/ThinkStats2-master/code

/Users/emilyvincett/Downloads/ThinkStats2-master/code


In [2]:
import pandas as pd
import numpy as np 
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import thinkstats2
import warnings 
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

# Read requisite files into program

In [3]:
# The data being analysed is campaign data from Facebook's Ad Manager
# File : Delivery of campaign by demographics (i.e. gender & age)
# Dev : Device (iphone, android etc), platform (facebook, insta, audience network etc)
# Time : Time of day in hours
# Due to the incongruency of how Facebook provides this data, merging them all into one set is
# not a viable means of assessment. My analysis involve looking at each one individually and building i
# inference from each.


file = pd.read_csv('/Users/emilyvincett/Downloads/File (1).csv')
dev = pd.read_csv('/Users/emilyvincett/Downloads/Dev (1).csv')
time = pd.read_csv('/Users/emilyvincett/Downloads/Time (1).csv')

# Arranges the sales data by time of day in ascending order 
# i.e from 12 midnight onwards

time.sort_values(by='Time of Day (Ad Account Time Zone)')

,Campaign Name,Time of Day (Ad Account Time Zone),Impressions,Frequency,Currency,Amount Spent (GBP),Landing Page Views,Cost per Landing Page View,Purchases,Cost per Purchase,"Cost per 1,000 People Reached","CPM (Cost per 1,000 Impressions)",Reporting Starts,Reporting Ends
160,Dig Distribution (Global),00:00:00 - 00:59:59,65,NaN,GBP,0.59,1.0,0.590,NaN,NaN,NaN,9.076923,2020-12-01,2021-02-25
94,Conversions: 16 - 34/Distribution,00:00:00 - 00:59:59,133,NaN,GBP,0.94,1.0,0.940,NaN,NaN,NaN,7.067669,2020-12-01,2021-02-25
184,Dig Distribution (Global) - Lifetime Budget,00:00:00 - 00:59:59,33,NaN,GBP,0.53,2.0,0.265,NaN,NaN,NaN,16.060606,2020-12-01,2021-02-25
419,Conversions 29,00:00:00 - 00:59:59,11,NaN,GBP,0.16,NaN,NaN,NaN,NaN,NaN,14.545455,2020-12-01,2021-02-25
458,Conversions: PageView Lookalike,00:00:00 - 00:59:59,19,NaN,GBP,0.11,NaN,NaN,NaN,NaN,NaN,5.789474,2020-12-01,2021-02-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,Conversions: Buyers Lookalike,23:00:00 - 23:59:59,58,NaN,GBP,0.34,1.0,0.340,NaN,NaN,NaN,5.862069,2020-12-01,2021-02-25
329,Conversions: Jan 28 - Copy,23:00:00 - 23:59:59,27,NaN,GBP,0.29,2.0,0.145,NaN,NaN,NaN,10.740741,2020-12-01,2021-02-25
308,Conversions 29,23:00:00 - 23:59:59,30,NaN,GBP,0.32,NaN,NaN,NaN,NaN,NaN,10.666667,2020-12-01,2021-02-25
455,Conversions - Prev Link Clicks - Copy,23:00:00 - 23:59:59,22,NaN,GBP,0.11,NaN,NaN,NaN,NaN,NaN,5.000000,2020-12-01,2021-02-25


In [4]:
# Cleans wrong purchase data in each file. 
# For some reason the FB pixel fired off more times than actual sales suggest.

file.fillna(0,inplace=True)
dev.fillna(0,inplace=True)
time.fillna(0,inplace=True)
file.shape, dev.shape,time.shape

((325, 14), (429, 18), (543, 14))

In [5]:
# This simple function converts categorical columns into numbers based on their rank.
# With rank determined by mean purchases in ascending order.
# So rank 0 is the lowest rank. 

def replaces(dataframe,col):
    arranged = dataframe.groupby(col)['Purchases'].mean().sort_values()
    for a in enumerate(arranged.index.values):
        dataframe.replace({col:{a[1]:a[0]}},inplace=True)
        

In [6]:
# Removes unneccessary columns
time.drop(['Reporting Starts','Reporting Ends'],axis=1,inplace=True)

In [7]:
#Time Index 
#for a in ['Campaign Name','Time of Day (Ad Account Time Zone)']:
 #   print(time.groupby(a)['Purchases'].mean().sort_values().to_frame().reset_index())

In [8]:
# Uses the replaces function created earlier and feeds it 
# a list of columns to rank based on mean purchases
# essential converting them from categories into numbers for further analysis

for a in ['Campaign Name','Time of Day (Ad Account Time Zone)']:
    replaces(time,a)
    
# Renaming specific columns into shorter names for my convenience
    
time.rename({'Campaign Name':'Campaign',
             'Time of Day (Ad Account Time Zone)':'Time',
            'Amount Spent (GBP)':'AmountSpent',
            'Landing Page Views':'LandingPageView',
            'Cost per Landing Page View':'CPPLV',
            'Cost per Purchase':'CPP',
            'Cost per 1,000 People Reached':'CP1000R',
            'CPM (Cost per 1,000 Impressions)':'CPM1000'},axis=1,inplace=True)

In [9]:
# Analysis of variance - generalised t - test conducted on each column 
# Using Purchases as the output variable and AmountSpent as a hardcoded input
# The code below feeds all the columns in the file time into the function 
# Creates a model consisting of the AmountSpent and each column seperately
# to explain the level of variance experienced in Purchases
# The higher the explained variance, the better a predictor that column would be.
# results are stored in the list t.

t = []
for name in time.columns:
    try:
        formula = 'Purchases ~ AmountSpent +' +name
        model = smf.ols(formula,data=time)
        results = model.fit()
    except (ValueError, TypeError):
        continue
    t.append((results.rsquared,name))    

In [10]:
# The list is reversed to show explained variance in decreasing order
# and converted into a dataframe evt for readability.
# The dataframe shows that viewing campaign data based on the time of day that ads ran 
# cost per purchase is the strongest predictor of sales when considering amountspent on ads
# landingpageview is a distant 2nd, makes sense all buyers would have to land on the page to buy

t.sort(reverse=True)
evt = pd.DataFrame(t[1:],columns=['Explained Variance','Features'])
evt.reset_index(inplace=True)
evt

,index,Explained Variance,Features
0,0,0.167314,CPP
1,1,0.053027,LandingPageView
2,2,0.052028,Time
3,3,0.040735,Campaign
4,4,0.027613,Impressions
5,5,0.025366,CPM1000
6,6,0.024812,CPPLV
7,7,0.024490,Frequency
8,8,0.024490,Currency
9,9,0.024490,CP1000R


In [11]:
# Renaming columns for my convenience

dev.rename({'Campaign Name':'Campaign',
           'Impression Device':'ImpressionDevice',
           'Device Platform':'DevicePlatform',
           'Amount Spent (GBP)':'AmountSpent',
           'Landing Page Views':'LandingPageViews',
           'Cost per Landing Page View':'CPLPV',
           'Cost per Purchase':'CPP',
           'Cost per 1,000 People Reached':'CP1000R',
           'CPM (Cost per 1,000 Impressions)':'CPM1000'},axis=1,inplace=True)

# Dropping unneccesary columns 
dev.drop(['Reporting Starts','Reporting Ends'],axis=1,inplace=True)

In [12]:
#Dev Index 
cols=['Campaign','ImpressionDevice','Platform','Placement','DevicePlatform']
#for a in cols:
 #   print(dev.groupby(a)['Purchases'].mean().sort_values().to_frame().reset_index())

In [13]:
# Uses the fuction created earlier replaces to convert categorical into numerical rank

for a in cols:
    replaces(dev,a)

In [14]:
# Analysis of variance - generalised t - test conducted on each column 
# Using Purchases as the output variable and AmountSpent as a hardcoded input
# The code below feeds all the columns in the file time into the function 
# Creates a model consisting of the AmountSpent and each column seperately
# to explain the level of variance experienced in Purchases
# The higher the explained variance, the better a predictor that column would be.
# results are stored in the list u.

u = []
for name in dev.columns:
    try:
        formula = 'Purchases ~ AmountSpent +' +name
        model = smf.ols(formula, data=dev)
        results = model.fit()
    except (ValueError, TypeError):
        continue
    u.append((results.rsquared,name))

In [15]:
# The list is reversed to show explained variance in decreasing order
# and converted into a dataframe evt for readability.
# The dataframe shows that viewing campaign data based on the time of day that ads ran 
# cost per purchase is the strongest predictor of sales when considering amountspent on ads
# specific campaigns is a distant 2nd, makes sense different campaigns have focused on difference 
# demographics, platforms and schedules which would impact what device and platform facebook 
# runs my ads on. 

u.sort(reverse=True)
evd = pd.DataFrame(u[1:],columns=['Explained Variance','Feature'])
evd.reset_index(inplace=True)
evd

,index,Explained Variance,Feature
0,0,0.195797,CPP
1,1,0.112297,Campaign
2,2,0.107129,CPLPV
3,3,0.091319,Platform
4,4,0.091080,ImpressionDevice
5,5,0.090917,Reach
6,6,0.090867,LandingPageViews
7,7,0.090752,Placement
8,8,0.090553,DevicePlatform
9,9,0.089992,CP1000R


In [16]:
# Rename columns for my convenience

file.rename({'Campaign Name':'Campaign',
             'Amount Spent (GBP)':'AmountSpent',
             'Landing Page Views':'LandingPageViews',
             'Cost per Landing Page View':'CPLPV',
             'Cost per Purchase':'CPP'},axis=1,inplace=True)
file.drop(['Reporting Starts','Reporting Ends'],axis=1,inplace=True)

In [17]:
# Index
cols_b = ['Campaign','Age','Gender']
#for a in cols_b:
 #   print(file.groupby([a])['Purchases'].mean().sort_values().to_frame().reset_index())

In [18]:
# Uses the replaces function to convert categorical columns into numerical ranks

for a in cols_b:
    replaces(file,a)

In [19]:
# Analysis of variance - generalised t - test conducted on each column 
# Using Purchases as the output variable and AmountSpent as a hardcoded input
# The code below feeds all the columns in the file time into the function 
# Creates a model consisting of the AmountSpent and each column seperately
# to explain the level of variance experienced in Purchases
# The higher the explained variance, the better a predictor that column would be.
# results are stored in the list v.

v = []
for name in file.columns:
    try:
        formula = 'Purchases ~ AmountSpent +' +name
        model = smf.ols(formula, data=file)
        results = model.fit()
    except (ValueError, TypeError):
        continue
    v.append((results.rsquared,name))

In [20]:
# The list is reversed to show explained variance in decreasing order
# and converted into a dataframe evt for readability.
# The dataframe shows that viewing campaign data based on the time of day that ads ran 
# campaign is the strongest predictor of sales when considering amountspent on ads
# cost per purchase is a distant 2nd, makes sense different campaigns have focused on difference 
# demographics, platforms and schedules which would impact what device and platform facebook 
# runs my ads on. 


v.sort(reverse=True)
evf = pd.DataFrame(v[1:],columns=['Explained Variance','Feature'])
evf.reset_index(inplace=True)
evf

,index,Explained Variance,Feature
0,0,0.192113,Campaign
1,1,0.183979,CPP
2,2,0.171254,Impressions
3,3,0.167271,Reach
4,4,0.165756,Frequency
5,5,0.165585,Age
6,6,0.165417,Gender
7,7,0.165245,LandingPageViews
8,8,0.165232,CPLPV
9,9,0.165229,Currency


# Combines the explained variances from the 3 different files into 1

In [21]:
combi = evf.merge(evd,left_on='index',right_on='index').merge(evt,left_on='index',right_on='index')
combi.rename({'Explained Variance_x':'File',
             'Feature_x':'File Features',
             'Explained Variance_y':'Device',
             'Feature_y':'Device Features',
             'Explained Variance':'Time',
             'Features':'Time Features'},axis=1,inplace=True)
combi.drop('index',axis=1,inplace=True)
combi

,File,File Features,Device,Device Features,Time,Time Features
0,0.192113,Campaign,0.195797,CPP,0.167314,CPP
1,0.183979,CPP,0.112297,Campaign,0.053027,LandingPageView
2,0.171254,Impressions,0.107129,CPLPV,0.052028,Time
3,0.167271,Reach,0.091319,Platform,0.040735,Campaign
4,0.165756,Frequency,0.091080,ImpressionDevice,0.027613,Impressions
5,0.165585,Age,0.090917,Reach,0.025366,CPM1000
6,0.165417,Gender,0.090867,LandingPageViews,0.024812,CPPLV
7,0.165245,LandingPageViews,0.090752,Placement,0.024490,Frequency
8,0.165232,CPLPV,0.090553,DevicePlatform,0.024490,Currency
9,0.165229,Currency,0.089992,CP1000R,0.024490,CP1000R


# Logistics Regression 

In [33]:
# I am going to build a logistics regression to model purchase data.
# Using a logistics regression bypasses the normality assumption needed for linear regression.
# Purchases are not normally distributed. 

# Codes 1 where a purchase took place and 0 where it didn't.
file['Bought'] = np.where(file['Purchases']>0,1,0)
time['Bought'] = np.where(time['Purchases']>0,1,0)
dev['Bought'] = np.where(dev['Purchases']>0,1,0)

# My regression model will attempt to build a probablistic model to 
# predict the likelihood of a purchase using a range of predictors.

# Model 1

In [41]:
formula = 'Bought ~ Campaign>27 + LandingPageViews>2 + AmountSpent>1 + Gender==2 + Age'
model1 = smf.logit(formula, data=file)
results1 = model1.fit()
results1.summary()

Optimization terminated successfully.
         Current function value: 0.073955
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 Bought   No. Observations:                  325
Model:                          Logit   Df Residuals:                      319
Method:                           MLE   Df Model:                            5
Date:                Sat, 20 Mar 2021   Pseudo R-squ.:                  0.4999
Time:                        14:55:42   Log-Likelihood:                -24.035
converged:                       True   LL-Null:                       -48.057
Covariance Type:            nonrobust   LLR p-value:                 3.480e-09
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       -5.7367      2.025     -2.832      0.005      -9.707      -1.767
Campaign > 27[T.True]            2.5550      0.887      2.879      0.004       0.816       4.294
LandingPageViews > 2[T.True]     5.5200      1.538      3.588      0.000       2.505       8.535
AmountSpent > 1[T.True]         -1.8997      1.253     -1.516      0.129      -4.355       0.556
Gender == 2[T.True]             -0.4196      0.877     -0.478      0.632      -2.139       1.300
Age                             -0.0649      0.379     -0.171      0.864      -0.808       0.678
================================================================================================
"""

# Model 2

In [42]:
dev['Bought'] = np.where(dev['Purchases']>0,1,0)
formula = 'Bought ~ Campaign>27 + Platform>2 + ImpressionDevice>4 + Placement==11'
model2 = smf.logit(formula, data=dev)
results2 = model2.fit()
results2.summary()

Optimization terminated successfully.
         Current function value: 0.078204
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 Bought   No. Observations:                  429
Model:                          Logit   Df Residuals:                      424
Method:                           MLE   Df Model:                            4
Date:                Sat, 20 Mar 2021   Pseudo R-squ.:                  0.3442
Time:                        14:56:34   Log-Likelihood:                -33.549
converged:                       True   LL-Null:                       -51.157
Covariance Type:            nonrobust   LLR p-value:                 4.196e-07
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       -8.1321      1.425     -5.707      0.000     -10.925      -5.339
Campaign > 27[T.True]            3.1198      0.738      4.229      0.000       1.674       4.566
Platform > 2[T.True]             2.0624      1.093      1.887      0.059      -0.080       4.205
ImpressionDevice > 4[T.True]     2.0020      0.756      2.647      0.008       0.520       3.484
Placement == 11[T.True]          1.0137      0.712      1.424      0.154      -0.381       2.409
================================================================================================
"""

# Model 3

In [43]:
time['Bought'] = np.where(time['Purchases']>0,1,0)
formula = 'Bought ~ Campaign>27 + Time>15 + LandingPageView>3 + AmountSpent>2.1'
model3 = smf.logit(formula, data=time)
results3 = model3.fit()
results3.summary()

Optimization terminated successfully.
         Current function value: 0.051453
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 Bought   No. Observations:                  543
Model:                          Logit   Df Residuals:                      538
Method:                           MLE   Df Model:                            4
Date:                Sat, 20 Mar 2021   Pseudo R-squ.:                  0.4396
Time:                        14:57:06   Log-Likelihood:                -27.939
converged:                       True   LL-Null:                       -49.853
Covariance Type:            nonrobust   LLR p-value:                 6.969e-09
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                      -8.1563      1.402     -5.819      0.000     -10.903      -5.409
Campaign > 27[T.True]           2.5068      0.921      2.722      0.006       0.702       4.312
Time > 15[T.True]               3.6687      1.229      2.985      0.003       1.259       6.078
LandingPageView > 3[T.True]     2.1210      0.936      2.266      0.023       0.286       3.955
AmountSpent > 2.1[T.True]       1.2163      0.934      1.303      0.193      -0.613       3.046
===============================================================================================
"""

# Accuracy of the 3 diffrent models

In [71]:
models = {model1:results1,
              model2:results2,
              model3:results3}
model = 0
for a,b in models.items():
    endog = pd.DataFrame(a.endog,columns=[a.endog_names])
    actual = endog['Bought']
    predict = (b.predict()>=actual.mean())
    true_pos = predict*actual
    true_neg = (1-predict)*(1-actual)
    acc = (sum(true_pos)+sum(true_neg))/len(actual)
    model +=1
    print('Model: ' +str(model)+', Accuracy: %.2f%%' %( acc*100))

Model: 1, Accuracy: 89.23%
Model: 2, Accuracy: 83.92%
Model: 3, Accuracy: 93.74%


# Conclusion

In [ ]:
# All 3 models have some predictability on Purchases but the dataset broken up by the time of day
# seems to be the most accurate 
# I would use this information to schedule my ads in the most profitable times based on Purchases and 
# Impression Costs.